<a href="https://colab.research.google.com/github/Milafreire/projeto_extract_datasus_to_bigquery_with_mongodb_pyspark/blob/main/Extracao_datasus_to_mongodb_atlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [37]:
#Credentials

In [13]:
import os
import requests
from requests.auth import HTTPBasicAuth
from pymongo import MongoClient, errors

In [33]:
def get_mongo_client():
    try:
        client = MongoClient(connection_url)
        return client
    except errors.ConnectionError as e:
        print(f"Erro ao conectar ao MongoDB: {e}")
        raise

In [34]:
def get_auth_token(auth_url, username, password):
    headers = {'Content-Type': 'application/json'}
    try:
        response = requests.get(auth_url, auth=HTTPBasicAuth(username, password), headers=headers)
        response.raise_for_status()
        return response.headers.get('Authorization')
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter o token de autenticação: {e}")
        raise

In [28]:
def fetch_data(data_url, username, password):
    headers = {'Content-Type': 'application/json'}
    data_payload = {"size": "10000"}
    try:
        response = requests.post(data_url, json=data_payload, auth=HTTPBasicAuth(username, password), headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados: {e}")
        raise

In [35]:
def insert_data_to_mongo(collection, data):
    try:
        collection.insert_many(data['hits']['hits'], ordered=False)
        print("Dados inseridos no MongoDB com sucesso!")
    except errors.BulkWriteError as e:
        print(f"Erro ao inserir dados no MongoDB: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")

In [38]:
def main():
    # Conectar ao MongoDB
    client = get_mongo_client()
    db = client[database_name]
    collection = db[collection_name]

    # Obter token de autenticação
    token = get_auth_token(url_name, username, password)

    # Buscar dados
    data = fetch_data(url_data, username, password)

    # Inserir dados no MongoDB
    insert_data_to_mongo(collection, data)

if __name__ == "__main__":
    main()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

